In [17]:
import pandas as pd
import os

# Define file paths
data_file = 'C:/Users/LENOVO/Desktop/Actor Facial Recognition/archive/labels.csv'
labels_file = 'C:/Users/LENOVO/Desktop/Actor Facial Recognition/archive/labels.csv'

# Check if files exist
if not os.path.exists(data_file):
    raise FileNotFoundError(f"Data file not found: {data_file}")

if not os.path.exists(labels_file):
    raise FileNotFoundError(f"Labels file not found: {labels_file}")

# Load the CSV data
data = pd.read_csv(data_file)
labels = pd.read_csv(labels_file, header=None, names=['emotion'])

# Display the first few rows of each CSV
print("Data File:")
print(data.head())

print("\nLabels File:")
print(labels.head())

# Prepare emotion labels mapping
emotion_labels = labels['emotion'].tolist()
emotion_labels_dict = {i: label for i, label in enumerate(emotion_labels)}

# Print emotion labels dictionary
print("\nEmotion Labels Dictionary:")
print(emotion_labels_dict)

# Example function to map numerical labels to emotion names
def get_emotion_label(numeric_label):
    return emotion_labels_dict.get(numeric_label, 'Unknown')

# Example usage of the function
example_numeric_label = 0
print(f"\nExample Emotion for label {example_numeric_label}: {get_emotion_label(example_numeric_label)}")


Data File:
   Unnamed: 0                     pth     label    relFCs
0           0  anger/image0000006.jpg  surprise  0.873142
1           1  anger/image0000060.jpg     anger  0.852311
2           2  anger/image0000061.jpg     anger  0.800957
3           3  anger/image0000066.jpg   disgust  0.843079
4           4  anger/image0000106.jpg     anger  0.849108

Labels File:
                                                    emotion
NaN     pth                    label                 relFCs
0.0     anger/image0000006.jpg surprise  0.8731421290934949
1.0     anger/image0000060.jpg anger      0.852310783018639
2.0     anger/image0000061.jpg anger       0.80095684657714
3.0     anger/image0000066.jpg disgust   0.8430792768021197

Emotion Labels Dictionary:
{0: 'relFCs', 1: '0.8731421290934949', 2: '0.852310783018639', 3: '0.80095684657714', 4: '0.8430792768021197', 5: '0.8491084661862777', 6: '0.8194483454798387', 7: '0.8520518574751808', 8: '0.8414882854744647', 9: '0.8231325554226703', 10:

In [18]:
#1. Image Loading and Preprocessing

In [20]:
import os
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your data
data_file = 'C:/Users/LENOVO/Desktop/Actor Facial Recognition/archive/labels.csv'
data = pd.read_csv(data_file)

# Map labels to numerical values if necessary
emotion_labels = ['anger', 'contempt', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
emotion_labels_dict = {i: label for i, label in enumerate(emotion_labels)}
label_mapping = {label: idx for idx, label in emotion_labels_dict.items()}
data['label'] = data['label'].map(label_mapping)

# Preprocessing functions
def load_image(image_path, target_size=(224, 224)):
    """
    Load and preprocess an image.
    
    Parameters:
        image_path (str): Path to the image file.
        target_size (tuple): Desired image size (width, height).
        
    Returns:
        np.ndarray: Preprocessed image as a NumPy array.
    """
    try:
        image = Image.open(image_path)
        image = image.resize(target_size)
        image_array = np.array(image)
        return image_array
    except FileNotFoundError:
        print(f"File not found: {image_path}")
        return np.zeros((target_size[0], target_size[1], 3))

def preprocess_images(dataframe, image_column, root_dir, target_size=(224, 224)):
    """
    Preprocess all images in a DataFrame.
    
    Parameters:
        dataframe (pd.DataFrame): DataFrame containing image paths and labels.
        image_column (str): Column name with image paths.
        root_dir (str): Root directory of the images.
        target_size (tuple): Desired image size (width, height).
        
    Returns:
        np.ndarray: Array of preprocessed images.
    """
    images = []
    for i in range(len(dataframe)):
        image_path = dataframe.iloc[i][image_column]
        full_image_path = os.path.join(root_dir, image_path.replace('\\', '/'))  # Ensures forward slashes
        images.append(load_image(full_image_path, target_size))
    return np.array(images)

# Specify the root directory where the images are stored
root_dir = 'C:/Users/LENOVO/Desktop/Actor Facial Recognition/archive'

# Prepare the images and labels
X = preprocess_images(data, 'pth', root_dir)
y = np.array(data['label'])

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
#2. Model Creation and Training

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense

# Define the model
model = Sequential([
    Input(shape=(224, 224, 3)),  # Define input shape
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(8, activation='softmax')  # Adjust the number of output units as per your label classes
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 8)                   │           1,032 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,169,992 (42.61 MB)

 Trainable params: 11,169,992 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#3. Evaluating Model Performance

In [24]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

177/177 ━━━━━━━━━━━━━━━━━━━━ 67s 376ms/step - accuracy: 0.1111 - loss: 15.7374
Validation Loss: 15.703718185424805
Validation Accuracy: 0.11783496290445328


In [25]:
#4. Making Predictions

In [36]:
def predict_emotion(model, image_path, target_size=(224, 224)):
    """
    Predict the emotion of an image using the trained model.
    
    Parameters:
        model (tf.keras.Model): Trained CNN model.
        image_path (str): Path to the image file.
        target_size (tuple): Desired image size (width, height).
        
    Returns:
        str: Predicted emotion label.
    """
    image = load_image(image_path, target_size)
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    predictions = model.predict(image)
    predicted_class = np.argmax(predictions, axis=1)[0]
    return emotion_labels_dict[predicted_class]

# Example usage
example_image_path = 'C:/Users/LENOVO/Desktop/Actor Facial Recognition/archive/disgust/image0000197.jpg'
predicted_emotion = predict_emotion(model, example_image_path)
print(f"Predicted Emotion: {predicted_emotion}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Predicted Emotion: neutral


In [30]:
model.save('C:/Users/LENOVO/Desktop/Actor Facial Recognition/facialRecognition_model.keras')